# Coursera IBM Data Science Professional Certificate
### Applied Data Science Capstone Project Week 3
The following notebook is the code and mardown for the IBM Data Science Professional Certificate Applied Data Science
- Explore, segment, and cluster the neighborhoods in the city of Toronto. 


## Import Dependencies 

In [1]:
import pandas as pd
import numpy as np
from config import client_id, client_secret
import requests

## Assign URL to variable

In [2]:
post_codes_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

# Use BeautifulSoup to read in data

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(post_codes_url,'lxml')

## Loop through data and assign Table Data HTML tags to empty lists

In [4]:
post_codes = []
boroughs = []
neighborhoods = []
counter = 1
for i in soup.table.find_all('td'):
    if counter == 1:
        post_codes.append(i.text)
    if counter == 2:
        boroughs.append(i.text)
    if counter == 3:
        neighborhoods.append(i.text)
    counter += 1
    if counter == 4:
        counter = 1 

## Create DataFrame and assign lists to columns

In [5]:
toronto_df = pd.DataFrame(columns=['PostalCode','Borough','Neighborhood'])
toronto_df['PostalCode'] = post_codes
toronto_df['Borough'] = boroughs
toronto_df['Neighborhood'] = neighborhoods
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


## Clean up DataFrame by getting rid of 'Not Assigned" Boroughs and getting rid of the '\n' newlines

In [6]:
toronto_df = toronto_df[toronto_df['Borough'] != 'Not assigned']
toronto_df.reset_index(inplace=True)
toronto_df['Neighborhood'] = toronto_df['Neighborhood'].str.replace("\n","")
toronto_df.head()

,index,PostalCode,Borough,Neighborhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,Harbourfront
3,5,M5A,Downtown Toronto,Regent Park
4,6,M6A,North York,Lawrence Heights


## Get the Shape of the DataFrame

In [7]:
toronto_df.shape

(211, 4)

## Get Geo Data

In [8]:
geo_df = pd.read_csv('https://cocl.us/Geospatial_data')
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merge DataFrames

In [9]:
toronto_df_merged = pd.merge(toronto_df,geo_df,left_on="PostalCode",\
                             right_on="Postal Code",how='left').drop('Postal Code',axis=1)
del toronto_df_merged['index']
toronto_df_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763
